In [1]:
# Copyright 2021 Google
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Time Crystal Data Collection

This notebook acts as a script to run the experiments and save the data associated with Figures 2d through 3d in the paper: Observation of Time-Crystalline Eigenstate Order on a Quantum Processor ([Nature](https://www.nature.com/articles/s41586-021-04257-w)). 

Each of the five experiments are built using `recirq.time_crystals.dtctasks.CompareDTCTask`, which defines the experiment parameters that are to be compared. `CompareDTCTask.dtctasks()` then creates `recirq.time_crystals.dtctasks.DTCTask`s with all of the requisite parameters for an instance of the experiment. 

A `DTCTask` has the following attributes and default values: 
- `qubits`: Sequence of qubits connected in a chain. Defaults to a line of $16$ connected `cirq.GridQubits`. 
- `disorder_instances`: Number of disorder instances to simulate and average resulting polarizations over. Defaults to $36$.
- `g`: Control parameter which influences how well the system is able to maintain time-crystalline behavior. Used in `cirq.PhasedXZGate`s in the circuit. Defaults to $0.94$.
- `initial_state` or `initial_states`: Only one should be supplied. Defines the input state of the system and is implemented with `cirq.Y` gates in the circuit. If `initial_state` is supplied, it will be repeated and used for every disorder instance. Defaults to a different random bit string for each disorder instance.
- `local_fields`: Random potentials critical to enable many-body local behavior. Used in `cirq.PhasedXZGate`s in the circuit. Defaults to uniformly selected float values between $-1.0$ and $1.0$.
- `thetas`, `zetas` and `chis`: Parameters used in the FSim gates in the circuit. Defaults to zero in all cases.
- `phis`: Parameter used in the FSim gates in the circuit. Affects the stability of the time-crystalline behavior. Defaults to uniformly selected float values between $-1.5*\pi$ and $-0.5*\pi$. 
- `gammas`: Parameter used in the FSim gates in the circuit. `Gammas` and `phis` are interdependent such that they satisfy $gammas = -2*phis$; One is set according to this equation if the other is supplied, otherwise use the default `phis` and calculate `gammas` from that.

`CompareDTCTask.dtctasks()` takes the product over the values of the `options_dict` supplied to the `CompareDTCTask` object, and passes those values as parameters to the `__init__()` function for `DTCTask`. Any parameter not supplied takes it's default value, meaning only the different parameter options that are to be compared need to be supplied to `CompareDTCTask`'s `options_dict`. This also means that supplying one parameter option for a parameter fixes that parameter option across all cases, instead of using the defaults, which may randomly generate values for each different `DTCTask`.  

## Setup

In [ ]:
!pip install cirq --pre --quiet
try:
    import recirq
except ImportError:
    !pip install --quiet git+https://github.com/quantumlib/ReCirq

In [3]:
import cirq
import itertools
import numpy as np
import matplotlib.pyplot as plt
import recirq.time_crystals as time_crystals

## Variables used in all experiments
Defines the qubits, number of DTC cycles (time steps) to evaluate, and the `base_dir` to store experiment results in.

In [4]:
# define the qubits to use
qubit_locations = [(3, 9), (3, 8), (3, 7), (4, 7), (4, 8), (5, 8), (5, 7), (5, 6), (6, 6), (6, 5), (7, 5), (8, 5),
              (8, 4), (8, 3), (7, 3), (6, 3)]

qubits = [cirq.GridQubit(*idx) for idx in qubit_locations]
num_qubits = len(qubits)

# number of cycles to evaluate over
num_cycles = 100

# directory to store data in
base_dir = time_crystals.DEFAULT_BASE_DIR

## Figure 2d's Experiment
This experiment considers the constant `g`, which affects the ability for the DTC system to oscillate consistently.

It compares two different values for `g`, $0.6$ and $0.94$, but uses the same disorder instances (randomly selected parameter values) for each of the two values of `g`. 

Define this with `options_dict` below, which results takes a product over the values of the dictionary. The result is the following two `recirq.time_crystals.DTCTasks`, with different values of `g`, but using the same `initial_states`, `local_fields`, and `gammas`: 
- `DTCTask(g = 0.6, initial_states = initial_states, local_fields = local_fields, gammas = gammas)`
- `DTCTask(g = 0.94, initial_states = initial_states, local_fields = local_fields, gammas = gammas)`

These two `DTCTask`s are each simulated over 36 disorder instances, have the polarizations for each qubit calculated, autocorrelated with the initial state, averaged, and finally saved as a json. 

In [5]:
%%time

# number of disorder instances to average over
disorder_instances = 36

# disorder instances h
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# initial states, one for each disorder instance
initial_states = np.random.choice(2, (disorder_instances, num_qubits))

# gammas for phased FSim gates
gammas = np.random.uniform(-0.5*np.pi, -1.5*np.pi, (disorder_instances, num_qubits - 1))

# create comparison task
options_dict = {
    'g': [0.6, 0.94],
    'initial_states': [initial_states], 
    'local_fields' : [local_fields],
    'gammas': [gammas]
}
comparedtctask = time_crystals.CompareDTCTask(qubits, num_cycles, disorder_instances, options_dict)

# create polarizations generator
polarizations_generator = time_crystals.run_comparison_experiment(comparedtctask, autocorrelate=True, take_abs=False)

# collect polarizations by g option
average_polarizations = np.empty((2, num_cycles+1, num_qubits))

for g_index, disorder_averaged_polarizations in enumerate(polarizations_generator):
    average_polarizations[g_index, :, :] = disorder_averaged_polarizations

# save data in json format
filename = f'{base_dir}/2d.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 2min 48s, sys: 84.9 ms, total: 2min 48s
Wall time: 2min 48s


## Figure 3a's Experiment
This experiment compares six options: the product between:
- Two options for `phis`: Uniformly, randomly selected `phis` between $-1.5\pi$ and $-0.5\pi$, and a fixed value of $-0.4$ for all `phis`. 
- Three options for `initial_state`: 
    - The polarized state of all zeros: `0000000000000000`
    - The Néel state of alternating zeros and ones: `0101010101010101`
    - A state with randomly selected zeros and ones: `00111000010011001111` (the first $16$ qubits)

It uses the same `local_fields` for all cases, autocorrelates the polarizations relative to the initial states, and averages over $24$ disorder instances. 

It also averages the `disorder_averaged_polarizations` over all $16$ qubit states before storing the results.

In [6]:
%%time

# number of disorder instances to average over
disorder_instances = 24

# disorder instances h
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# prepare 3 initial states to compare
neel_initial_state = np.tile([0,1], num_qubits//2)
polarized_initial_state = np.full(num_qubits, 0)
random_initial_state = [0,0,1,1,1,0,0,0,0,1,0,0,1,1,0,0,1,1,1,1][:num_qubits]
initial_states = [neel_initial_state, polarized_initial_state, random_initial_state]

# prepare random and fixed phis to compare
disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

# create comparison task
options_dict = {
    'local_fields': [local_fields], 
    'initial_state': initial_states,
    'phis': [disordered_phis, fixed_phis]
}
options_order = ['local_fields', 'phis', 'initial_state']
comparedtctask = time_crystals.CompareDTCTask(qubits, num_cycles, disorder_instances, options_dict, options_order)

# prepare polarizations and indices generators
polarizations_generator = time_crystals.run_comparison_experiment(comparedtctask, autocorrelate=True, take_abs=False)
indices_iterator = itertools.product(range(2), range(len(initial_states)))

# collect polarizations averaged over qubit sites by phi and initial state options
average_polarizations = np.empty((2, len(initial_states), num_cycles+1))

for (phi_index, initial_state_index), disorder_averaged_polarizations in zip(indices_iterator, polarizations_generator):
    # store average over all qubit sites
    average_polarizations[phi_index, initial_state_index, :] = np.mean(disorder_averaged_polarizations, axis=1)

# save data in json format
filename = f'{base_dir}/3a.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 5min 37s, sys: 140 ms, total: 5min 37s
Wall time: 5min 37s


## Figure 3b's Experiment
This experiment compares $40$ different cases, the product between: 
- Two options for `phis`: Uniformly, randomly selected `phis` and `phis` fixed at $-0.4$.
- 20 options for `initial_state`: 20 randomly selected bit string initial states.

Again, the same random potentials, `local_fields`, are used in all cases. The resulting polarizations are autocorrelated with the initial states, **have their absolute value taken**, and are averaged over $24$ disorder instances. 

This time, store the average over all of the $16$ qubits, but only for cycles $30$ and $31$. 

In [7]:
%%time

# instance counts for disorder and random initial states
disorder_instances = 24
initial_state_instances = 20 # this is 500 in the paper

# disorder instances h
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# prepare random initial states
initial_states = np.random.choice(2, (initial_state_instances, num_qubits))

# prepare random and fixed phis to compare
disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

# create comparison task
options_dict = {
    'local_fields': [local_fields],
    'initial_state': initial_states,
    'phis': [disordered_phis, fixed_phis]
}
options_order = ['local_fields', 'phis', 'initial_state']
comparedtctask = time_crystals.CompareDTCTask(qubits, num_cycles, disorder_instances, options_dict, options_order)

# prepare polarizations and indices generators
polarizations_generator = time_crystals.run_comparison_experiment(comparedtctask, autocorrelate=True, take_abs=True)
indices_iterator = itertools.product(range(2), range(len(initial_states)))

# collect polarizations, averaged over qubit sites and cycles 30 and 31, by phi and initial state options
average_polarizations = np.empty((2, initial_state_instances))

for (phi_index, initial_state_index), disorder_averaged_polarizations in zip(indices_iterator, polarizations_generator):
    # store average over qubit sites and cycles 30 and 31
    average_polarizations[phi_index, initial_state_index] = np.mean(disorder_averaged_polarizations[30:32, :])
    
# save data in json format
filename = f'{base_dir}/3b.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 37min 47s, sys: 925 ms, total: 37min 48s
Wall time: 37min 48s


## Figure 3c's Experiment
This experiment compares $4$ different cases, the product between: 
- Two options for `phis`: Uniformly, randomly selected `phis` and `phis` fixed at $-0.4$.
- Two options for `initial_state`: 
    - The polarized initial state of all zeros: `0000000000000000`
    - The polarized initial state, but disturbed at qubit index $11$: `0000000000010000`

The same `local_fields`, are used in all cases. The polarizations are **not** autocorrelated, and are averaged over $24$ disorder instances. 

Store the polarizations matrix of shape `(num_cycles + 1, num_qubits)` without averaging, but only over cycles $30$ through $60$, and only over qubits $11$ through $14$.  

In [8]:
%%time

# number of disorder instances to average over
disorder_instances = 24

# disorder parameters h
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# prepare random and fixed phis to compare
disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

# prepare two initial states to compare
polarized_initial_state = [0]*num_qubits
disturb_qubit = 11
disturbed_polarized_initial_state = list(polarized_initial_state)
disturbed_polarized_initial_state[disturb_qubit] = 1
initial_states = [polarized_initial_state, disturbed_polarized_initial_state]

# create comparison task
options_dict = {
    'local_fields': [local_fields],
    'initial_state': initial_states,
    'phis': [disordered_phis, fixed_phis]
    
}
options_order = ['local_fields', 'phis', 'initial_state']
comparedtctask = time_crystals.CompareDTCTask(qubits, num_cycles, disorder_instances, options_dict, options_order)

# prepare polarizations and indices generators
polarizations_generator = time_crystals.run_comparison_experiment(comparedtctask, autocorrelate=False, take_abs=False)
indices_iterator = itertools.product(range(2), range(len(initial_states)))

# collect polarizations, averaged over cycles 30 through 60 and qubits 11 through 14, by phi and initial state options
average_polarizations = np.empty((2, 2, 31, 4))

for (phi_index, initial_state_index), disorder_averaged_polarizations in zip(indices_iterator, polarizations_generator):
    # store average over cycles 30 and 31, and qubits 11 through 14
    average_polarizations[phi_index, initial_state_index, :, :] = disorder_averaged_polarizations[30:61, 11:15]

# save data in json format
filename = f'{base_dir}/3c.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 3min 44s, sys: 108 ms, total: 3min 44s
Wall time: 3min 44s


## Figure 3d's Experiment
This again experiment compares $4$ different cases, the product between: 
- Two options for `phis`: Uniformly, randomly selected `phis` and `phis` fixed at $-0.4$.
- Two options for `initial_state`: 
    - The polarized initial state of all zeros: `0000000000000000`
    - The polarized initial state, but disturbed at qubit index $11$: `0000000000010000`

However, this experiment differs from 3c's in that, for each of the two `initial_state`s, different random `local_fields` and `phis` are generated. Do this by creating two different `CompareDTCTasks` for the different `initial_state`s, each of which compares `phis`. The collected polarizations are **not** autocorrelated, and are averaged over $24$ disorder instances. 

Store the polarizations matrix of shape `(num_cycles + 1, num_qubits)` without averaging, indexed by `phis` option and `initial_state` option. 

In [9]:
%%time

# prepare two initial states to compare
polarized_initial_state = np.full(num_qubits, 0)
disturb_qubit = 11
disturbed_polarized_initial_state = polarized_initial_state.copy()
disturbed_polarized_initial_state[disturb_qubit] = 1
initial_states = [polarized_initial_state, disturbed_polarized_initial_state]

# use different disorder instances for the two initial states
disorder_instances_options = [64, 81]

# collect polarizations by phi and initial state options
average_polarizations = np.empty((2, 2, num_cycles + 1, num_qubits))

# iterate over initial states and their associated number of disorder instances
for initial_state_index, (initial_state, disorder_instances) in enumerate(zip(initial_states, disorder_instances_options)): 
    
    # disorder parameter h
    local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))
    
    # prepare random and fixed phis to compare
    disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
    fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

    # create comparison task
    options_dict = {
        'initial_state': [initial_state],
        'local_fields': [local_fields],
        'phis': [disordered_phis, fixed_phis]
    }
    options_order = ['local_fields', 'phis', 'initial_state']
    comparedtctask = time_crystals.CompareDTCTask(qubits, num_cycles, disorder_instances, options_dict, options_order)

    # prepare polarizations and indices generators
    polarizations_generator = time_crystals.run_comparison_experiment(comparedtctask, autocorrelate=False, take_abs=False)
    
    for phi_index, disorder_averaged_polarizations in enumerate(polarizations_generator):
        # store average polarizations
        average_polarizations[phi_index, initial_state_index, :, :] = disorder_averaged_polarizations

# save data in json format
filename = f'{base_dir}/3d.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 11min 25s, sys: 272 ms, total: 11min 25s
Wall time: 11min 25s


## Next Steps
With the data collected and saved, move on to the [Time Crystal Data Analysis](time_crystal_data_analysis.ipynb) notebook to generate the plots and evaluate their results.